### Shared Word

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 200  # 28  # native word end index

# word sample
word_sample = True  # True, False
word_sample_num = 20

# sentence check
twogram_sentence_check = False  # True, False
threegram_sentence_check = False

# shared word frequency
shared_word_frequency = True  # True, False

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word
etymology_word = True  # True for adding etymology word

# adding output file extention
if (not prefix_suffix) & etymology_word & native_word:
    file_ext = "1"
elif (not prefix_suffix) & etymology_word & (not native_word):
    file_ext = "2"
elif prefix_suffix & etymology_word & native_word:
    file_ext = "3"
elif prefix_suffix & etymology_word & (not native_word):
    file_ext = "4"
elif prefix_suffix & (not etymology_word) & native_word:
    file_ext = "5"
elif (not prefix_suffix) & (not etymology_word) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              
# 1 => for native word and etymology word without prefix suffix. 
# 2 => for only etymology word without prefix suffix. 
# 3 => for native word and etymology word with prefix suffix. prefix_suffix, native_word and etymology_word must be True. 
# 4 => for only etymology word with prefix suffix.
# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"{file_ext}")

1


In [4]:
Path(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/4-Shared Word/{lang_folder.capitalize()} {lang_pair.capitalize()}").mkdir(parents=True, exist_ok=True)

In [5]:
def remove_repetition(word_group):
    '''
    remove_repetition(word_group): detect word repetion in word group 
    '''
    words = word_tokenize(word_group)
    word_unique = set(words)
    if len(word_unique) == 1:
        return "repetitive_word_group"
    else:
        return word_group

In [6]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,you,102069964
1,i,94447074
2,the,77481215
3,to,58281119
4,is,50852895
...,...,...
552149,fruitcocktail,6
552150,andthesunlightshining,6
552151,upravo,6
552152,yagawa,6


In [7]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [8]:
# Option
if prefix_suffix:
    df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_{word_end}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
    df_word = df_word.loc[:,["word","frequency"]]
    df_word = pd.concat([df_word,df_word_select], axis=0)
    df_word.drop_duplicates(inplace=True)    
    df_word.sort_values(by="frequency", ascending=False, inplace=True)
    df_word.reset_index(drop=True, inplace=True)
else:
    df_word = df_word_select

if native_word:
    df_word
else:
    df_word = df_word.head(0)

df_word

,word,frequency
0,you,102069964
1,i,94447074
2,the,77481215
3,to,58281119
4,is,50852895
5,a,49726799
6,not,48510772
7,it,47050281
8,that,35204210
9,and,34806253


In [9]:
#lang_pair_list = glob.glob(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()}_And_{lang_pair.lower().capitalize()}*_All.xlsx")
#lang_pair_list

In [10]:
#df_pair = pd.read_excel(f"{lang_pair_list[0]}")
df_pair = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.lower().capitalize()}/{lang_folder.capitalize()}_{lang_pair.lower().capitalize()}_Shared_Vocabulary.xlsx")
#df_pair = df_pair.head()
df_pair

,dict_entry_main,turkish_word
0,abacus,abaküs
1,blockade,abluka
2,absorb,absorbe
3,absurd,absürt
4,azalea,açelya
...,...,...
1773,zebra,zebra
1774,zigzag,zikzak
1775,zombie,zombi
1776,zoology,zooloji


In [11]:
# Option
if prefix_suffix:
    df_prefix_suffix_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Word_Prefix_Suffix_Custom_Result.xlsx")
    df_prefix_suffix_select = df_prefix_suffix_select.loc[:,["search_word","word"]]
    df_prefix_suffix_select.rename(columns={"search_word":"dict_entry_main"}, inplace=True)
    df_pair = pd.merge(df_pair,df_prefix_suffix_select, how="inner", on="dict_entry_main")
    df_pair.drop_duplicates(inplace=True)
    df_pair.reset_index(drop=True, inplace=True)
    df_pair = df_pair.loc[:,["word",f"{lang_pair.lower()}_word"]]
    df_pair.rename(columns={"word":"dict_entry_main"}, inplace=True)
else:
    pass

if etymology_word:
    df_pair
else:
    df_pair = df_pair.head(0)
    
df_pair

,dict_entry_main,turkish_word
0,abacus,abaküs
1,blockade,abluka
2,absorb,absorbe
3,absurd,absürt
4,azalea,açelya
...,...,...
1773,zebra,zebra
1774,zigzag,zikzak
1775,zombie,zombi
1776,zoology,zooloji


In [12]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]

In [13]:
word_select = df_word["word"].values.tolist()
words = df_pair["dict_entry_main"].values.tolist()
word_select_set = set(word_select)
disable_word_set = set(disable_list)
words_set = set(words)

In [14]:
word_list = list(word_select_set.union(words_set.difference(disable_word_set)))

In [15]:
# 2 way  Not: 1 way is long and in Shared Word ReDe part 
data_kind_list = ["Two", "Three"]
for i in data_kind_list:
    sent_ngram_file = f"{i}" # Two, Three, Four, Five, Sentence
 
    if sent_ngram_file.lower() != "sentence":
        data_kind = f"{sent_ngram_file}gram"  # Twogram, Threegram, Fourgram, Fivegram
        folder_path = "N Gram"
    else:
        data_kind = "sentence"
        folder_path = "Sentence"

    df = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/{folder_path.capitalize()}/Merge/{data_kind.capitalize()}_Merge.csv")
    d_list  = df.iloc[:,0].values.tolist()

    resultlist = []
    manager = multiprocessing.Manager()
    resultlist = manager.list()
    
    def word_in_wordgroup(d_list):
        mergelist = []
        try:
            word = d_list.split()
        except:
            word = []
            #pass  disabled for non split value
        var1 = range(len(word))
        for j in var1:
            if word[j] in word_list:
                mergelist.append(word[j])
                if len(mergelist) == len(word):
                        resultlist.append(d_list)
                            
    if __name__ == '__main__':
        # with Pool(16) as p:
        with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
            p.map(word_in_wordgroup, d_list) # string_word liste

    result_list = list(resultlist)
    df_result = pd.DataFrame(result_list, columns=[0])  # add columns parameter for empty result
    df_result = df_result.rename(columns = {0: f"{data_kind.lower()}"})
    df_merge = pd.merge(df_result, df, how="left", on=f"{data_kind.lower()}")
    df_merge_result = df_merge.sort_values(by="frequency", ascending=False)
    df_merge_result.drop_duplicates(inplace=True)
    df_merge_result[f"{data_kind.lower()}"] = df_merge_result[f"{data_kind.lower()}"].apply(lambda x: remove_repetition(x))  # added remove_repetition 
    df_merge_result = df_merge_result[df_merge_result[f"{data_kind.lower()}"] != "repetitive_word_group"]  # remove_repetition part
    df_merge_result.reset_index(drop=True, inplace=True)

    df_merge_result.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_{data_kind.capitalize()}{file_ext}.xlsx", index=False)    

#### Concat Result

In [16]:
df_word = pd.DataFrame(word_list, columns=["word"])
df_word

,word
0,mafia
1,cafe
2,national
3,illusion
4,rating
...,...
1762,blocked
1763,mademoiselle
1764,emperor
1765,amber


In [17]:
# Option For Frequency
if shared_word_frequency:
    df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
    df_word = pd.merge(df_word,df_word_all, how="inner", on="word")
    df_word.drop_duplicates(inplace=True)
    df_word.sort_values(by="frequency", ascending=False, inplace=True)
    df_word.reset_index(drop=True, inplace=True)
else:
    pass

df_word

,word,frequency
0,you,102069964
1,i,94447074
2,the,77481215
3,to,58281119
4,is,50852895
...,...,...
2312,he,6
2313,wow,6
2314,this,6
2315,you,6


In [18]:
df_shared_twogram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Twogram{file_ext}.xlsx")
df_shared_twogram = df_shared_twogram.loc[:,["twogram","frequency"]]
df_shared_twogram.rename(columns={"frequency":"freq_twogram"}, inplace=True)
df_shared_twogram

,twogram,freq_twogram
0,i am,10494331
1,do not,8972296
2,i do,4179914
3,i will,3706224
4,do you,3358227
...,...,...
78354,quality level,4
78355,contact sector,4
78356,quality index,4
78357,pop baby,4


In [19]:
# Option For Twogram Sentence
if twogram_sentence_check:
    df_twogram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Two_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_twogram_sent.rename(columns={"two_gram":"twogram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_twogram_sent = df_twogram_sent.loc[:,["twogram"]]
    df_shared_twogram = pd.merge(df_shared_twogram, df_twogram_sent, how="inner", on="twogram")
    df_shared_twogram.drop_duplicates(inplace=True)
    df_shared_twogram.reset_index(drop=True, inplace=True)
else:
    pass

df_shared_twogram


,twogram,freq_twogram
0,i am,10494331
1,do not,8972296
2,i do,4179914
3,i will,3706224
4,do you,3358227
...,...,...
78354,quality level,4
78355,contact sector,4
78356,quality index,4
78357,pop baby,4


In [20]:
df_shared_threegram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Threegram{file_ext}.xlsx")
df_shared_threegram = df_shared_threegram.loc[:,["threegram","frequency"]]
df_shared_threegram.rename(columns={"frequency":"freq_threegram"}, inplace=True)
df_shared_threegram

,threegram,freq_threegram
0,i do not,3520257
1,i am not,1262941
2,what do you,1120086
3,you do not,1010676
4,do not you,644488
...,...,...
206638,i control radiation,4
206639,the antibiotic i,4
206640,my radar screen,4
206641,cup will not,4


In [21]:
# Option For Threegram Sentence
if threegram_sentence_check:
    df_threegram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Three_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_threegram_sent.rename(columns={"three_gram":"threegram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_threegram_sent = df_twogram_sent.loc[:,["threegram"]]
    df_shared_threegram = pd.merge(df_shared_threegram, df_threegram_sent, how="inner", on="threegram")
    df_shared_threegram.drop_duplicates(inplace=True)
    df_shared_threegram.reset_index(drop=True, inplace=True)
else:
    pass

df_shared_threegram

,threegram,freq_threegram
0,i do not,3520257
1,i am not,1262941
2,what do you,1120086
3,you do not,1010676
4,do not you,644488
...,...,...
206638,i control radiation,4
206639,the antibiotic i,4
206640,my radar screen,4
206641,cup will not,4


In [27]:
df_shared_all = pd.concat([df_word, df_shared_twogram, df_shared_threegram], axis=1)
df_shared_all

,word,frequency,twogram,freq_twogram,threegram,freq_threegram,fourgram,freq_fourgram,fivegram,freq_fivegram,sentence,freq_sentence
0,you,102069964.0,i am,10494331.0,i do not,3520257,you do not have,160426.0,you do not have to,107450.0,of course,266277.0
1,i,94447074.0,do not,8972296.0,i am not,1262941,i do not have,158788.0,i do not have a,34568.0,what is it,251203.0
2,the,77481215.0,i do,4179914.0,what do you,1120086,do not have to,150328.0,do not have to do,21572.0,stop it,152280.0
3,to,58281119.0,i will,3706224.0,you do not,1010676,no i do not,105374.0,i do not have to,21499.0,i am,136931.0
4,is,50852895.0,do you,3358227.0,do not you,644488,i do not like,99318.0,i do not like it,18564.0,i do,114801.0
...,...,...,...,...,...,...,...,...,...,...,...,...
206638,NaN,NaN,NaN,NaN,i control radiation,4,NaN,NaN,NaN,NaN,NaN,NaN
206639,NaN,NaN,NaN,NaN,the antibiotic i,4,NaN,NaN,NaN,NaN,NaN,NaN
206640,NaN,NaN,NaN,NaN,my radar screen,4,NaN,NaN,NaN,NaN,NaN,NaN
206641,NaN,NaN,NaN,NaN,cup will not,4,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
df_shared_all.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Twogram_Thereegram_Result_With_Frequency{file_ext}.xlsx", index=False)

##### Concat Result With Comma

In [29]:
def word_in_wordgroup(df, source_column, target_column):

    '''word_in_wordgroup(df, "word", "twogram"):
       df is dataframe, source_column and target_column are 
       dataframe column string name. source_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_word_result = pd.DataFrame()
    for i in df[f"{source_column}"].dropna():
        try:
            if word_sample:
                word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)  # Option
            else:
                word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)] 
        except:
            pass        
        word_in_word_cluster.insert(0,f"{source_column}",i)
        df_word_result = pd.concat([df_word_result,word_in_word_cluster], axis=0)
    df_word_result.reset_index(drop=True, inplace=True)

    return df_word_result

In [30]:
df_word_order_twogram = word_in_wordgroup(df_shared_all, "word", "twogram")
df_word_order_threegram = word_in_wordgroup(df_shared_all, "word", "threegram")  

In [31]:
df_word_order_join_twogram = df_word_order_twogram.groupby(["word"])["twogram"].apply(", ".join).reset_index()   # df_word_order_11.groupby(["word"])[f"twogram_pair_{lang_pair.lower()}"].transform(lambda x: ','.join(x))
df_word_order_join_threegram = df_word_order_threegram.groupby(["word"])["threegram"].apply(", ".join).reset_index()

In [32]:
dfs = [df_word_order_join_twogram,df_word_order_join_threegram]

In [33]:
df_word_order_join_all = reduce(lambda  left,right: pd.merge(left,right, on=['word'], how='outer'), dfs)  # left,right make left to right merge
#df_word_order_join_all = reduce(lambda  right,left: pd.merge(left,right, on=['word'], how='outer'), dfs)  # right,left make right to left merge
df_word_order_join_all  

,word,twogram,threegram,fourgram,fivegram,sentence
0,a,"have a, is a, was a, for a, in a, not a, like ...","i am a, i have a, this is a, you have a, it wa...","i am not a, do not have a, do you have a, this...","i do not have a, you do not have a, we do not ...","a what, not a chance, what a surprise, i am a ..."
1,abacus,"the abacus, abacus to, your abacus, abacus is,...","like the abacus, abacus to the, and the abacus...",i like the abacus,NaN,"and the abacus, my abacus"
2,abnormal,"the abnormal, no abnormal, abnormal psychology...","i am abnormal, in the abnormal, am i abnormal,...","contact in the abnormal, and you have abnormal...","we have on the abnormal, the reaction to the a...","abnormal psychology, i am abnormal, am i abnor..."
3,absorb,"to absorb, absorb the, absorb it, will absorb,...","to absorb the, will absorb the, and absorb it,...","it and absorb it, will absorb the radiation, t...","that will absorb the radiation, the ocean will...","absorb it, to absorb, i absorb, absorb this in..."
4,absurd,"is absurd, absurd to, the absurd, this absurd,...","this is absurd, that is absurd, it is absurd, ...","absurd is not it, absurd your position is, you...","is it not absurd to, absurd name for a cricket...","this is absurd, that is absurd, it is absurd, ..."
...,...,...,...,...,...,...
1695,zebra,"a zebra, the zebra, is zebra, that zebra, acti...","this is zebra, like a zebra, to action zebra, ...","the zebra and the, i am a zebra, i am the zebr...","that i was a zebra, i am not a zebra, the zebr...","a zebra, this is zebra, i am a zebra, the zebr..."
1696,zigzag,"the zigzag, a zigzag, to zigzag, negative zigz...","am the zigzag, we will zigzag, in a zigzag, of...","i am the zigzag, i zigzag like this",NaN,"negative zigzag, a zigzag"
1697,zombie,"a zombie, the zombie, zombie is, your zombie, ...","like a zombie, not a zombie, am a zombie, is a...","i am a zombie, am not a zombie, is not a zombi...","i am not a zombie, zombie what do we do, have ...","a zombie, i am a zombie, i am not a zombie, yo..."
1698,zoology,"the zoology, in zoology, zoology department, o...","the zoology department, this zoology module, d...","a doctor of zoology, the zoology and the, a do...","not a doctor of zoology, a doctor in zoology n...","i am not a doctor of zoology, in zoology i am ..."


In [34]:
# Option For Word Frequency
if shared_word_frequency:
    df_word_order_join_all = pd.merge(df_word_order_join_all,df_word_all, how="inner", on="word")
    df_word_order_join_all.drop_duplicates(inplace=True)
    df_word_order_join_all = df_word_order_join_all.loc[:,["word","frequency","twogram","threegram","fourgram","fivegram","sentence"]]
    df_word_order_join_all.sort_values(by="frequency", inplace=True, ascending=False)
    df_word_order_join_all.reset_index(drop=True, inplace=True)
else:
    pass

df_word_order_join_all

,word,frequency,twogram,threegram,fourgram,fivegram,sentence
0,you,102069964,"do you, you have, you do, not you, you will, f...","what do you, you do not, do not you, you have ...","you do not have, do you have a, no you do not,...","you do not have to, you do not have a, you do ...","and you, do you, you do, no you do not, you do..."
1,i,94447074,"i am, i do, i will, i have, i was, and i, what...","i do not, i am not, i am a, i have to, i will ...","i do not have, no i do not, i do not like, i a...","i do not have a, i do not have to, i do not li...","i am, i do, i will, i do not, i am not, no i d..."
2,the,77481215,"in the, to the, on the, of the, what the, for ...","this is the, i am the, out of the, it was the,...","i am in the, i am not the, this is not the, do...","i do not have the, i do not like the, i am not...","what the, the what, the police, the baby, the ..."
3,to,58281119,"have to, to the, to me, to do, to you, you to,...","i have to, you have to, we have to, not have t...","do not have to, you will have to, do i have to...","you do not have to, do not have to do, i do no...","i have to, you do not have to, you have to, to..."
4,is,50852895,"this is, is not, is it, what is, is that, it i...","what is it, this is not, this is a, this is th...","this is not a, is that what you, this is not t...","you have to do is, is not that what you, is th...","what is it, what is this, what is that, it is,..."
...,...,...,...,...,...,...,...
2299,of,6,"of the, out of, of course, of a, of you, of my...","out of the, of course i, out of my, of course ...","of course i am, i am out of, no of course not,...","in the name of the, of course you do not, of c...","of course, of course not, no of course not, of..."
2300,camp,6,"the camp, to camp, base camp, this camp, camp ...","in the camp, to the camp, we will camp, in thi...","this is base camp, me to the camp, the camp wa...","this for a band camp, this is a training camp,...","the camp, base camp, what camp, a camp, in the..."
2301,he,6,"he was, is he, he is, he will, and he, what he...","he was a, he will not, he was not, is not he, ...","he was in the, he was not a, no he will not, o...","he will not do it, he was shot in the, he is i...","he is, he was, is he, he will, he will not, he..."
2302,do,6,"do not, i do, do you, you do, what do, to do, ...","i do not, what do you, you do not, do not you,...","you do not have, i do not have, do not have to...","you do not have to, i do not have a, do not ha...","i do, do not, i do not, do it, do you, you do,..."


In [35]:
df_word_order_join_all.word.nunique()

1700

In [36]:
df_word_order_join_all.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Twogram_Threegram_Join_Result_Without_Frequency{file_ext}.xlsx", index=False)

#### Copy Move And Delete

In [37]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Twogram_Threegram*{file_ext}.xlsx")
output_file

['English_Turkish_Shared_Twogram1.xlsx',
 'English_Turkish_Shared_Threegram1.xlsx',
 'English_Turkish_Shared_Fourgram1.xlsx',
 'English_Turkish_Shared_Fivegram1.xlsx',
 'English_Turkish_Shared_Sentence1.xlsx',
 'English_Turkish_Shared_Result_With_Frequency1.xlsx',
 'English_Turkish_Shared_Join_Result_Without_Frequency1.xlsx']

In [38]:
for k in output_file:
    source = k # source directory
    destination = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/4-Shared Word/{lang_folder.capitalize()} {lang_pair.capitalize()}"
    shutil.copy2(source, destination)

In [39]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass